In [1]:
from qiskit import QuantumCircuit
from scipy.optimize import minimize
from scipy.optimize import minimize
from qiskit.providers.fake_provider import GenericBackendV2
import random
from itertools import product
import csv
import random
import math
from qiskit_ibm_runtime import QiskitRuntimeService

In [2]:

# Set up Qiskit Runtime service
service = QiskitRuntimeService(
    channel="ibm_quantum",
    instance="rpi-rensselaer/general/general",

)

# Get backend
rensslearBackend = service.backend("ibm_rensselaer")

In [3]:
# Takes a beta, gamma, depth, and adjacencyList; returns the QAOA circuit
def QAOA_circuit(beta: float, gamma: float, depth: int, adjacencyList: list[list[int]]):
    num_qubits = len(adjacencyList)
    # print(num_qubits)
    # print(adjacencyList)
    qc = QuantumCircuit(num_qubits, num_qubits)  # Allocate classical bits

    # Initial Hadamard gates
    qc.h(range(num_qubits))
    qc.barrier()

    for _ in range(depth):
        # Apply the Hamiltonian cost function
        for edgeIndex, neighbors in enumerate(adjacencyList):
            for edge in neighbors:
                if edge > edgeIndex:
                    qc.cz(edgeIndex, edge)
                    qc.rz(2 * gamma, edgeIndex)
                    qc.cz(edge, edgeIndex)
        qc.barrier()

        # Apply RX gates to each qubit for the mixer Hamiltonian
        for qubit in range(num_qubits):
            qc.rx(2 * beta, qubit)
        qc.barrier()

    # Measurement
    qc.measure(range(num_qubits), range(num_qubits))

    return qc

In [4]:
# Takes a beta, gamma, bool representing if the backend is fake, depth of the circuit, and adjacnecy list; returns a dict of the bitstrings produced
from qiskit import transpile


def QAOA(
    beta: float,
    gamma: float,
    fakeBackend: bool,
    depth: int,
    adjacencyList: list[int, list[int]],
) -> dict():
    if fakeBackend:
        backend = GenericBackendV2(len(adjacencyList))
    else:
        rensslearBackend = service.backend("ibm_rensselaer")
        backend = rensslearBackend
    qc = QAOA_circuit(beta, gamma, depth, adjacencyList)
    if not fakeBackend:
        #transpile the circuit
        qc = transpile(qc, backend)
        
    job = backend.run(qc, shots=1024)
    result = job.result()
    return result.get_counts()

In [5]:
# Takes params->[beta, gamma], a bool representing if the backend is fake, depth of the circuit, and adjacnecy list;
# returns the average cut of the given graph
def cost_function(
    params, fakeBackend: bool, depth: int, adj_list: list[int, list[int]]
):
    beta, gamma = params
    counts = QAOA(beta, gamma, fakeBackend, depth, adj_list)
    # Calculate the expectation value of the cost Hamiltonian
    cost = 0
    for bitstring, count in counts.items():
        bit_val = [int(bit) for bit in bitstring]
        cut_value = 0
        for i in range(len(adj_list)):
            for j in adj_list[i]:
                if i < j:
                    cut_value += bit_val[i] != bit_val[j]
        cost += cut_value * count
    return -cost / 1024

In [6]:
def optimize_qaoa_params(cost_function, initial_params, depth, adj_list, backend):
    # Optimization of the QAOA parameters
    result = minimize(
        cost_function, initial_params, args=(backend, depth, adj_list), method="COBYLA"
    )
    optimal_params = result.x
    optimal_value = -result.fun
    # Execute the final circuit with the optimal parameters
    final_counts = QAOA(optimal_params[0], optimal_params[1], backend, depth, adj_list)
    return {
        "optimal_params": optimal_params,
        "optimal_value": optimal_value,
        "final_counts": final_counts,
    }

In [7]:
def read_csv(filename):
    with open(filename, mode="r", newline="") as file:
        reader = csv.reader(file)
        for row in reader:
            print(row)

In [8]:
def csv_parser(filename):
    graphs = []
    with open(filename, mode="r", newline="") as file:
        # Make an array like this: [[graphid, adjlist], [graphid, adjlist], ...]
        reader = csv.reader(file)

        for row in reader:
            graphs.append(row[:2])
        # Remove the header
        graphs = graphs[1:]
        # Convert the graph[n][1] from a string into an adjacency list
        for i in range(len(graphs)):
            graphs[i] = graphs[i][1]
            # make it no longer a string
            graphs[i] = eval(graphs[i])
    return graphs

In [9]:
def new_csv_parser(filename):
    graphs = []
    betas = []
    gammas = []
    GCNGammas = []
    GCNBetas = []
    with open(filename, mode="r", newline="") as file:
        # Make an array like this: [[graphid, adjlist], [graphid, adjlist], ...]
        reader = csv.reader(file)

        for row in reader:
            graphs.append(row[:2])
            betas.append(row[2])
            gammas.append(row[3])
            GCNBetas.append(row[5])
            GCNGammas.append(row[6])
        # Remove the header
        graphs = graphs[1:]
        betas = betas[1:]
        gammas = gammas[1:]
        GCNBetas = GCNBetas[1:]
        GCNGammas = GCNGammas[1:]
        # Convert the graph[n][1] from a string into an adjacency list
        for i in range(len(graphs)):
            graph = graphs[i][1]
            # make it no longer a string
            graph = eval(graph)
            graphs[i] = [graph, float(betas[i]),float(gammas[i]),float(GCNBetas[i]),float(GCNGammas[i])]
    return graphs

In [10]:
# Read graphs from csv
filename = "testing.csv"
read_csv(filename)
# assign a variable to the csv
graphs = csv_parser(filename)

['Index', 'Graph ', 'Classical Beta', 'Classical Gamma', 'Classical EV', 'GCN Beta', 'GCN Gamma', 'GCN EV']
['1', '[[1, 12, 13, 8, 6, 4, 3], [11, 4, 9, 10, 0, 2, 3, 14, 12, 5], [8, 9, 1, 12, 10, 6, 14, 13], [14, 5, 12, 1, 10, 0], [10, 6, 1, 9, 5, 0], [3, 6, 8, 10, 14, 7, 4, 1, 12], [4, 5, 7, 13, 0, 2, 14], [9, 5, 6, 11], [5, 11, 2, 0, 12, 9], [1, 4, 7, 2, 14, 11, 13, 12, 8], [4, 1, 5, 3, 13, 2, 14], [1, 8, 14, 9, 13, 7, 12], [3, 13, 0, 2, 8, 1, 11, 9, 14, 5], [12, 0, 6, 14, 10, 9, 11, 2], [3, 11, 5, 9, 1, 13, 10, 2, 12, 6]]', '0.6722489910905410', '0.5702986488179130', '28.419921875', '-0.06553309', '0.11482838', '27.9755859375']
['0', '[[12, 16, 19, 11, 2, 1], [18, 15, 10, 12, 16, 4, 17, 0], [4, 19, 7, 9, 3, 0], [16, 7, 11, 15, 4, 8, 6, 2, 10, 18, 9], [2, 11, 5, 3, 8, 19, 1, 18, 13], [14, 13, 18, 19, 12, 8, 4, 11, 7, 6], [7, 15, 10, 3, 8, 5, 16], [6, 16, 2, 3, 14, 15, 8, 5], [12, 14, 17, 19, 5, 3, 6, 4, 18, 16, 7], [11, 10, 15, 12, 16, 2, 14, 3, 19], [9, 1, 6, 15, 3, 14], [9, 15, 3, 4

In [11]:
def optimize_qaoa_params(cost_function, initial_params, depth, adj_list, backend):
    # Optimization of the QAOA parameters
    result = minimize(
        cost_function, initial_params, args=(backend, depth, adj_list), method="COBYLA"
    )
    optimal_params = result.x
    optimal_value = -result.fun
    # Execute the final circuit with the optimal parameters
    final_counts = QAOA(optimal_params[0], optimal_params[1], backend, depth, adj_list)
    return {
        "optimal_params": optimal_params,
        "optimal_value": optimal_value,
        "final_counts": final_counts,
    }

In [12]:
import time

for i in range(len(graphs)):
    start = time.time()
    graph = graphs[i]
    initial_params = [0.5, 0.5]
    depth = 1
    print(graph)
    adj_list = graph
    backend = True
    result = optimize_qaoa_params(cost_function, initial_params, depth, adj_list, backend)
    print(result)
    end = time.time()
    print(end - start)

[[1, 12, 13, 8, 6, 4, 3], [11, 4, 9, 10, 0, 2, 3, 14, 12, 5], [8, 9, 1, 12, 10, 6, 14, 13], [14, 5, 12, 1, 10, 0], [10, 6, 1, 9, 5, 0], [3, 6, 8, 10, 14, 7, 4, 1, 12], [4, 5, 7, 13, 0, 2, 14], [9, 5, 6, 11], [5, 11, 2, 0, 12, 9], [1, 4, 7, 2, 14, 11, 13, 12, 8], [4, 1, 5, 3, 13, 2, 14], [1, 8, 14, 9, 13, 7, 12], [3, 13, 0, 2, 8, 1, 11, 9, 14, 5], [12, 0, 6, 14, 10, 9, 11, 2], [3, 11, 5, 9, 1, 13, 10, 2, 12, 6]]
{'optimal_params': array([0.70321923, 0.56504061]), 'optimal_value': 30.453125, 'final_counts': {'100111100111010': 1, '101101010011010': 1, '100001010111010': 1, '001101001011010': 1, '001001101111010': 1, '110101001101010': 1, '000011101010010': 1, '000101100111000': 2, '000011100011010': 1, '000101101001010': 1, '100101100010110': 1, '001101000011000': 1, '000011000001010': 1, '010111101011000': 1, '000001101010010': 1, '000101000010010': 1, '100001100111000': 1, '000001010111010': 1, '101101101111000': 1, '100001101011011': 1, '110101001111010': 1, '101101001001010': 1, '100

In [13]:
def find_beta_gammas(currentDataFilename, fakeBackend):
    graphs = new_csv_parser(currentDataFilename)
    print(graphs)
    # parse the csv file for graps beta and gamma values
    
    for graph in graphs:
        adj_list = graph[0]
        classicalBeta = graph[1]
        classicalGamma = graph[2]
        depth = 1  # Depth of the QAOA circuit
        classicalEV = abs(cost_function([classicalBeta, classicalGamma], fakeBackend, depth, adj_list))
        print("classical Optimizer", "Beta: ", classicalBeta, "Gamma: ", classicalGamma, "EV: ", classicalEV)
        # choose a random beta and gamma
        randomBeta = random.uniform(0, 2 * math.pi)
        randomGamma = random.uniform(0, 2 * math.pi)
        randomEV = abs(cost_function( [randomBeta, randomGamma], fakeBackend, depth, adj_list))
        print("random Optimizer: ", "Beta: ", randomBeta, "Gamma: ", randomGamma, "EV: ", randomEV)
        GCNBeta = graph[3]
        GCNGamma = graph[4]
        GCNEV = abs(cost_function([GCNBeta, GCNGamma], fakeBackend, depth, adj_list))
        print("GCN Optimizer: ", "Beta: ", GCNBeta, "Gamma: ", GCNGamma, "EV: ", GCNEV)
        print()
    return

In [ ]:
find_beta_gammas("testing.csv", False)

[[[[1, 12, 13, 8, 6, 4, 3], [11, 4, 9, 10, 0, 2, 3, 14, 12, 5], [8, 9, 1, 12, 10, 6, 14, 13], [14, 5, 12, 1, 10, 0], [10, 6, 1, 9, 5, 0], [3, 6, 8, 10, 14, 7, 4, 1, 12], [4, 5, 7, 13, 0, 2, 14], [9, 5, 6, 11], [5, 11, 2, 0, 12, 9], [1, 4, 7, 2, 14, 11, 13, 12, 8], [4, 1, 5, 3, 13, 2, 14], [1, 8, 14, 9, 13, 7, 12], [3, 13, 0, 2, 8, 1, 11, 9, 14, 5], [12, 0, 6, 14, 10, 9, 11, 2], [3, 11, 5, 9, 1, 13, 10, 2, 12, 6]], 0.672248991090541, 0.570298648817913, -0.06553309, 0.11482838], [[[12, 16, 19, 11, 2, 1], [18, 15, 10, 12, 16, 4, 17, 0], [4, 19, 7, 9, 3, 0], [16, 7, 11, 15, 4, 8, 6, 2, 10, 18, 9], [2, 11, 5, 3, 8, 19, 1, 18, 13], [14, 13, 18, 19, 12, 8, 4, 11, 7, 6], [7, 15, 10, 3, 8, 5, 16], [6, 16, 2, 3, 14, 15, 8, 5], [12, 14, 17, 19, 5, 3, 6, 4, 18, 16, 7], [11, 10, 15, 12, 16, 2, 14, 3, 19], [9, 1, 6, 15, 3, 14], [9, 15, 3, 4, 14, 5, 12, 0], [8, 14, 1, 5, 9, 0, 11], [5, 19, 18, 15, 4], [12, 5, 8, 7, 15, 11, 9, 10, 17], [6, 11, 1, 9, 3, 14, 10, 7, 18, 13], [17, 7, 3, 9, 1, 0, 8, 6], [1

/var/folders/5g/6nyzcp550497twzdzfnbl7300000gn/T/ipykernel_24689/3724340733.py:22: DeprecationWarning: backend.run() and related sessions methods are deprecated  as of qiskit-ibm-runtime 0.23 and will be removed no sooner than 6 months after the release date. More details can be found in the primitives migration guide https://docs.quantum.ibm.com/api/migration-guides/qiskit-runtime.
  job = backend.run(qc, shots=1024)


classical Optimizer Beta:  0.672248991090541 Gamma:  0.570298648817913 EV:  28.45703125
random Optimizer:  Beta:  5.443995520033808 Gamma:  6.104936431729077 EV:  28.48046875
GCN Optimizer:  Beta:  -0.06553309 Gamma:  0.11482838 EV:  27.9755859375

classical Optimizer Beta:  0.754627733579307 Gamma:  0.852306668680279 EV:  39.30859375
random Optimizer:  Beta:  2.5772497550203157 Gamma:  4.402113798300992 EV:  38.8515625
GCN Optimizer:  Beta:  -0.0340221 Gamma:  0.07576785 EV:  38.142578125

classical Optimizer Beta:  2.24133093696137 Gamma:  2.1869053864166 EV:  12.083984375
random Optimizer:  Beta:  1.9651667734107034 Gamma:  3.5623969635119725 EV:  12.0048828125
GCN Optimizer:  Beta:  -0.03197668 Gamma:  0.10506362 EV:  12.03515625

classical Optimizer Beta:  0.611211328006633 Gamma:  0.638159828056963 EV:  17.05078125
random Optimizer:  Beta:  0.07101053338025506 Gamma:  0.548561519986224 EV:  16.986328125
GCN Optimizer:  Beta:  -0.06295986 Gamma:  0.12024899 EV:  17.1962890625

cla